# LangSmith Tracing with v-router

This notebook demonstrates how to enable LangSmith tracing for multiple providers in v-router.

## Prerequisites

1. Install v-router with tracing support:
```bash
pip install 'v-router[tracing]'
```

2. Set up your LangSmith account and get your API key from [LangSmith](https://smith.langchain.com/)

## Set up environment variables

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Enable LangSmith tracing
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGSMITH_API_KEY")  # Your LangSmith API key

# Optional: Set a project name for better organization in LangSmith
os.environ["LANGCHAIN_PROJECT"] = "v-router-demo"

## Basic Usage with Tracing - OpenAI Example

In [ ]:
from v_router import Client, LLM

# Create a client with OpenAI provider
client = Client(
    llm_config=LLM(
        model_name="gpt-4o",
        provider="openai",
        max_tokens=1000,
        temperature=0.7
    )
)

# Make a simple request - this will be traced in LangSmith
response = await client.messages.create(
    messages=[
        {"role": "user", "content": "What is the capital of France?"}
    ]
)

print(response.content[0].text)

## Using Traceable Decorator with v-router

You can also use LangSmith's `@traceable` decorator to trace your own functions that use v-router:

In [ ]:
from langsmith import traceable

@traceable(name="Question Answering Pipeline")
async def answer_question(question: str) -> str:
    """Answer a question using v-router with OpenAI."""
    
    # This inner function call will also be traced
    response = await client.messages.create(
        messages=[
            {"role": "system", "content": "You are a helpful assistant. Answer concisely."},
            {"role": "user", "content": question}
        ]
    )
    
    return response.content[0].text

# Call the traced function
answer = await answer_question("What are the main ingredients in a margherita pizza?")
print(answer)

## Complex Example with Tool Use

In [ ]:
from v_router.classes.tools import Tool, Tools

@traceable(run_type="tool", name="Weather Lookup")
def get_weather(location: str) -> str:
    """Mock weather function for demonstration."""
    return f"The weather in {location} is sunny and 72°F."

@traceable(name="Weather Assistant")
async def weather_assistant(user_query: str):
    """Process weather queries with tool use."""
    
    # Define the weather tool
    weather_tool = Tool(
        name="get_weather",
        description="Get the current weather for a location",
        input_schema={
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA"
                }
            },
            "required": ["location"]
        }
    )
    
    tools = Tools(tools=[weather_tool])
    
    # First call to determine if we need to use a tool
    response = await client.messages.create(
        messages=[
            {"role": "user", "content": user_query}
        ],
        tools=tools,
        tool_choice="auto"
    )
    
    # Check if the model wants to use a tool
    if response.tool_use:
        tool_results = []
        for tool_use in response.tool_use:
            if tool_use.name == "get_weather":
                # Execute the tool (this is also traced)
                result = get_weather(**tool_use.arguments)
                tool_results.append({
                    "tool_use_id": tool_use.id,
                    "content": result
                })
        
        # Send tool results back to the model
        final_response = await client.messages.create(
            messages=[
                {"role": "user", "content": user_query},
                {"role": "assistant", "content": response.content},
                {"role": "user", "content": tool_results}
            ]
        )
        
        return final_response.content[0].text
    else:
        return response.content[0].text

# Run the weather assistant
result = await weather_assistant("What's the weather like in San Francisco?")
print(result)

## Viewing Traces in LangSmith

After running the above examples, you can view your traces in the LangSmith UI:

1. Go to [LangSmith](https://smith.langchain.com/)
2. Navigate to your project (default is "default" unless you set LANGCHAIN_PROJECT)
3. You'll see all your traced calls with:
   - Input/output data
   - Latency metrics
   - Token usage
   - Error tracking
   - Nested trace structure

## Disabling Tracing

To disable tracing, simply set the environment variable to false or remove it:

In [ ]:
# Disable tracing
os.environ["LANGCHAIN_TRACING_V2"] = "false"

# Or remove the environment variable entirely
# del os.environ["LANGCHAIN_TRACING_V2"]

## Example with Multiple Providers

LangSmith tracing works with all supported providers:

In [ ]:
# Re-enable tracing
os.environ["LANGCHAIN_TRACING_V2"] = "true"

# Example with Azure OpenAI
azure_client = Client(
    llm_config=LLM(
        model_name="gpt-4o",  # This should be your deployment name in Azure
        provider="azure",
        max_tokens=1000
    )
)

# This will also be traced
response = await azure_client.messages.create(
    messages=[
        {"role": "user", "content": "Write a haiku about cloud computing."}
    ]
)

print(response.content[0].text)

# Example with Anthropic (also traced)
anthropic_client = Client(
    llm_config=LLM(
        model_name="claude-3-5-sonnet-20241022",
        provider="anthropic",
        max_tokens=1000
    )
)

response = await anthropic_client.messages.create(
    messages=[
        {"role": "user", "content": "Explain quantum computing in one sentence."}
    ]
)

print(response.content[0].text)